In [ ]:
import torch
from torch.utils.data import Dataset
import os
import glob
from PIL import Image
import torchvision.transforms as transforms

# 路径

## 实现了自定义数据读取与格式转换，将原始图像数据进行归一化、尺寸调整和批量处理，生成适配模型训练的数据格式。同时，加载器支持多任务训练场景下曝光增强与检测任务的联合输入，保证训练数据在多GPU分布式训练中的高效调度与均衡分配。该模块为后续训练提供了稳定、标准化且高效的数据流，确保模型训练过程的顺利进行与性能稳定。

In [ ]:
# 导入 BrightISP.py 中的 HighExposure_Degrading 函数
# 确保相对于 load.py 的路径正确
import sys
# 将 utils 目录添加到系统路径以导入 DarkISP
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../utils')))
from BrightISP import HighExposure_Degrading

# 导入数据集

In [ ]:
class HighExposureDataset(Dataset):
    """
    自定义数据集类，用于从原始图像生成高曝光图像。
    它使用 DarkISP 中的 HighExposure_Degrading 函数来模拟高曝光条件。
    """

    def __init__(self, root_dir, transform=None, resize_size=(640, 640), target_subfolders=None):
        """
        初始化 HighExposureDataset。

        参数：
            root_dir (string)：包含所有图像的目录（例如，path/to/WIDER_train/images）。
            transform (callable, 可选)：可选的变换函数，应用于样本（在调整大小和降质后）。
            resize_size (tuple)：调整图像大小的目标尺寸（高度，宽度）。
            target_subfolders (list, 可选)：指定要加载的子文件夹前缀列表。
        """
        self.root_dir = root_dir  # 图像根目录
        self.transform = transform  # 图像变换
        self.resize_size = resize_size  # 调整大小的目标尺寸

        self.image_files = []  # 存储图像文件路径的列表
        all_subdirs = [d for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d))]  # 获取根目录下所有子目录

        if target_subfolders:
            # 如果指定了目标子文件夹前缀，则只加载匹配的子文件夹中的图像
            for subfolder_num_str in target_subfolders:
                # 查找以指定前缀开头的实际目录名称
                matched_dirs = [d for d in all_subdirs if d.startswith(subfolder_num_str + '--')]
                if matched_dirs:
                    # 假设只有一个目录匹配前缀
                    subfolder_path = os.path.join(self.root_dir, matched_dirs[0])
                    # 扩展图像文件列表，包含匹配子文件夹中的.jpg 和.png 图像
                    self.image_files.extend(glob.glob(os.path.join(subfolder_path, '*.jpg')))
                    self.image_files.extend(glob.glob(os.path.join(subfolder_path, '*.png')))
                else:
                    print(f"警告：在 {self.root_dir} 中未找到以 {subfolder_num_str}-- 开头的子目录")
        else:
            # 如果未指定目标子文件夹，则递归查找根目录下所有.jpg 和.png 图像
            self.image_files = glob.glob(os.path.join(self.root_dir, '**', '*.jpg'), recursive=True) \
                             + glob.glob(os.path.join(self.root_dir, '**', '*.png'), recursive=True)

        print(f"找到 {len(self.image_files)} 张图像。")
        
        # 定义调整大小的变换
        self.resize_transform = transforms.Resize(self.resize_size)

    def __len__(self):
        # 返回数据集中图像的数量
        return len(self.image_files)

    def __getitem__(self, idx):
        # 获取指定索引处的图像对（高曝光图像和原始图像）
        if torch.is_tensor(idx):
            # 如果索引是张量，转换为列表
            idx = idx.tolist()

        img_path = self.image_files[idx]  # 获取图像文件路径
        
        # 加载原始图像
        original_image = Image.open(img_path).convert('RGB')
        
        # 将 PIL 图像转换为张量并归一化到 [0, 1]
        original_tensor = transforms.ToTensor()(original_image)
        
        # 应用调整大小的变换
        original_tensor_resized = self.resize_transform(original_tensor)
        
        # 使用 DarkISP 函数在调整大小后的张量上生成高曝光图像
        # 该函数期望形状为 (C, H, W) 且范围在 [0, 1] 的张量
        high_exposure_tensor_resized, _ = HighExposure_Degrading(original_tensor_resized)
        
        # 如果指定了变换，应用变换
        if self.transform:
            original_tensor_resized = self.transform(original_tensor_resized)
            high_exposure_tensor_resized = self.transform(high_exposure_tensor_resized)

        # 返回调整大小后的高曝光图像（输入）和调整大小后的原始图像（真值）
        return high_exposure_tensor_resized, original_tensor_resized